In [1]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi

%matplotlib inline

In [2]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [3]:
dotenv.load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [5]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangm_tickers = ['AAPL', 'AMZN', 'FB', 'GOOG', 'MSFT', 'NFLX']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangm_original = alpaca.get_bars(faangm_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangm_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,118.73,118.8100,116.8600,117.34,34852374,187129,117.756760,AAPL
2015-12-02 05:00:00+00:00,117.00,118.1100,116.0800,116.28,33385643,180616,117.151198,AAPL
2015-12-03 05:00:00+00:00,116.55,116.7900,114.2200,115.20,41560785,245330,115.434888,AAPL
2015-12-04 05:00:00+00:00,115.20,119.2500,115.1100,119.03,57776977,307788,118.187290,AAPL
2015-12-07 05:00:00+00:00,118.98,119.8600,117.8100,118.28,32080754,190809,118.509111,AAPL
...,...,...,...,...,...,...,...,...
2022-04-04 04:00:00+00:00,374.00,396.0200,373.1300,391.50,7004417,145506,389.627400,NFLX
2022-04-05 04:00:00+00:00,389.46,390.0000,377.3050,380.15,4223477,110400,381.654450,NFLX
2022-04-06 04:00:00+00:00,374.15,376.4699,362.7300,368.35,4343830,134230,367.423997,NFLX


In [6]:
faangm = faangm_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol').fillna(0.0)
faangm.index = faangm.index.date
faangm.columns = faangm_tickers
faangm

,AAPL,AMZN,FB,GOOG,MSFT,NFLX
2015-12-01,117.34,679.06,107.12,767.04,55.22,125.37
2015-12-02,116.28,676.01,106.07,762.38,55.21,128.93
2015-12-03,115.20,666.25,104.38,752.54,54.20,126.81
2015-12-04,119.03,672.64,106.18,766.81,55.91,130.93
2015-12-07,118.28,669.83,105.61,763.25,55.81,125.36
...,...,...,...,...,...,...
2022-04-04,178.44,3366.93,233.89,2872.85,314.97,391.50
2022-04-05,175.06,3281.10,231.84,2821.26,310.88,380.15
2022-04-06,171.59,3175.12,223.30,2743.52,299.50,368.35
2022-04-07,172.14,3155.69,222.95,2729.30,301.37,362.15


In [7]:
kraken = krakenex.API(kraken_api_key, kraken_secret_key)

crypto_tickers = ['BTC', 'ETH', 'LUNA', 'SOL', 'XRP', 'ADA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'MATIC', 'DAI', 'LTC', 'ATOM', 'LINK', 'UNI', 'TRX', 'BCH']

kraken_tickers = ['BTCUSD', 'ETHUSD', 'LUNAUSD', 'SOLUSD', 'XRPUSD', 'ADAUSD', 'AVAXUSD', 'DOTUSD', 'DOGEUSD', 'SHIBUSD', 'MATICUSD', 'DAIUSD', 'LTCUSD', 'ATOMUSD', 'LINKUSD', 'UNIUSD', 'TRXUSD', 'BCHUSD']

pykraken = pykrakenapi.KrakenAPI(kraken)

dataframes = {}
for ticker in kraken_tickers:
    ticker_ohlc, last = pykraken.get_ohlc_data(ticker, 1440, 1500000000)
    ticker_ohlc['symbol'] = ticker
    dataframes[ticker] = ticker_ohlc.reset_index()
    time.sleep(1)

crypto_original = pd.DataFrame(pd.concat(dataframes.values(), join='inner')).set_index('dtime').sort_index()
crypto_original

,time,open,high,low,close,vwap,volume,count,symbol
dtime,,,,,,,,,
2020-04-20,1587340800,231.800000,238.600000,216.000000,219.100000,226.490000,1.017306e+04,2910,BCHUSD
2020-04-20,1587340800,0.012922,0.013125,0.012349,0.012446,0.012677,2.581566e+06,120,TRXUSD
2020-04-20,1587340800,0.189120,0.194870,0.178550,0.181770,0.186480,1.599498e+07,3393,XRPUSD
2020-04-20,1587340800,0.035500,0.036566,0.033921,0.033990,0.034962,6.669707e+06,669,ADAUSD
2020-04-20,1587340800,180.190000,186.760000,166.130000,170.490000,175.270000,1.724604e+05,18785,ETHUSD
...,...,...,...,...,...,...,...,...,...
2022-04-09,1649462400,3193.140000,3251.360000,3183.200000,3242.280000,3216.060000,1.238531e+04,9700,ETHUSD
2022-04-09,1649462400,110.730000,112.520000,110.160000,112.230000,111.150000,1.279938e+04,1948,LTCUSD
2022-04-09,1649462400,15.122320,15.460670,15.122320,15.371460,15.311320,3.301678e+04,1160,LINKUSD


In [8]:
crypto = crypto_original.drop(['time', 'open', 'high', 'low', 'vwap', 'volume', 'count'], axis = 1).pivot(columns = 'symbol').fillna(0.0)
crypto.index = crypto.index.date
crypto.columns = crypto_tickers
crypto

,BTC,ETH,LUNA,SOL,XRP,ADA,AVAX,DOT,DOGE,SHIB,MATIC,DAI,LTC,ATOM,LINK,UNI,TRX,BCH
2020-04-20,0.033990,2.2787,0.00,219.10,6839.2,1.01453,0.001960,0.0000,170.49,3.44413,40.32,0.00,0.0000,0.000000,0.00,0.012446,0.000,0.18177
2020-04-21,0.034635,2.2768,0.00,219.90,6852.1,1.01021,0.001984,0.0000,170.99,3.46270,40.65,0.00,0.0000,0.000000,0.00,0.012606,0.000,0.18340
2020-04-22,0.036665,2.3895,0.00,233.60,7136.9,1.01501,0.002005,0.0000,183.16,3.68806,41.83,0.00,0.0000,0.000000,0.00,0.012969,0.000,0.18815
2020-04-23,0.040810,2.4300,0.00,237.50,7485.0,1.01643,0.002060,0.0000,185.68,3.73123,42.95,0.00,0.0000,0.000000,0.00,0.013550,0.000,0.19283
2020-04-24,0.041764,2.7538,0.00,237.90,7507.8,1.01093,0.002084,0.0000,187.72,3.78000,44.51,0.00,0.0000,0.000000,0.00,0.013920,0.000,0.19328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-05,1.170137,29.2892,91.02,365.47,45479.2,0.99985,0.172689,21.7782,3407.70,16.74518,122.79,116.27,1.5938,0.000027,126.82,0.069272,11.122,0.81726
2022-04-06,1.053412,26.9086,82.81,331.19,43179.4,0.99987,0.143326,19.8483,3168.44,15.40933,111.79,108.10,1.4550,0.000024,113.02,0.063056,9.959,0.75979
2022-04-07,1.089906,27.7077,87.67,336.58,43469.8,0.99991,0.146052,20.4139,3229.00,15.76737,114.11,103.59,1.5123,0.000025,118.54,0.064429,10.452,0.78705
2022-04-08,1.025486,26.8784,83.83,323.16,42282.6,0.99993,0.142546,19.4176,3193.91,15.13765,110.76,94.58,1.4335,0.000024,109.84,0.062571,9.738,0.75525


In [9]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangm_caps_list = [2.804, 1.616, 0.60781, 1.81, 2.245, 0.163534]
faangm_cap_portfolio = pd.DataFrame({'tickers':faangm_tickers, 'market cap weights':[market_cap / sum(faangm_caps_list) for market_cap in faangm_caps_list]})
faangm_cap_portfolio

,tickers,market cap weights
0,AAPL,0.303255
1,AMZN,0.174772
2,FB,0.065735
3,GOOG,0.195753
4,MSFT,0.242799
5,NFLX,0.017686


In [10]:
mcw_faangm_daily_returns = pd.DataFrame(faangm.pct_change().dot(list(faangm_cap_portfolio.loc[:, 'market cap weights'])))
mcw_faangm_daily_returns.columns = ['faangm']
mcw_faangm_daily_returns

,faangm
2015-12-01,NaN
2015-12-02,-0.004900
2015-12-03,-0.013646
2015-12-04,0.024839
2015-12-07,-0.005089
...,...
2022-04-04,0.024245
2022-04-05,-0.017957
2022-04-06,-0.028908
2022-04-07,0.000003


In [11]:
random_crypto_portfolios = random_portfolios_generator(crypto_tickers, 10)
random_crypto_portfolios

,ticker,random weights 1,random weights 2,random weights 3,random weights 4,random weights 5,random weights 6,random weights 7,random weights 8,random weights 9,random weights 10
0,BTC,0.008562,0.011406,0.005344,0.119581,0.065175,0.117123,0.023254,0.048695,0.097544,0.046543
1,ETH,0.092280,0.039992,0.039539,0.039794,0.095326,0.040110,0.026700,0.090811,0.094043,0.026511
2,LUNA,0.105952,0.098556,0.002385,0.043059,0.010657,0.019556,0.008339,0.062930,0.078434,0.094588
3,SOL,0.016832,0.063982,0.131641,0.076395,0.007835,0.078298,0.073734,0.028056,0.032295,0.103286
4,XRP,0.041484,0.097105,0.132159,0.035405,0.074402,0.100132,0.033142,0.022450,0.000508,0.090237
5,ADA,0.017386,0.037598,0.019709,0.026552,0.059750,0.105352,0.010872,0.061369,0.020218,0.095764
6,AVAX,0.058723,0.076296,0.141320,0.019096,0.053576,0.004944,0.078065,0.032239,0.053998,0.025027
7,DOT,0.067477,0.063432,0.002954,0.054622,0.041182,0.041995,0.070978,0.004174,0.011942,0.096462
8,DOGE,0.023428,0.088680,0.030509,0.114741,0.018087,0.026195,0.033311,0.072725,0.034014,0.018479
9,SHIB,0.083257,0.025672,0.064534,0.024204,0.057765,0.009038,0.071995,0.058984,0.050311,0.035613


In [12]:
crypto_daily_returns_list = []
for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns_list.append(crypto.pct_change().dot(list(random_crypto_portfolios.loc[:, f'random weights {n + 1}'])))

crypto_daily_returns = pd.concat(crypto_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns = crypto_daily_returns.rename(columns = {n: f'random crypto portfolio {n + 1}'})

crypto_daily_returns

,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-05,-0.020209,-0.013931,0.000436,-0.025822,-0.015793,-0.021087,-0.011304,-0.021187,-0.019172,-0.021287
2022-04-06,-0.088983,-0.086562,-0.093564,-0.088175,-0.084643,-0.076079,-0.089983,-0.083772,-0.092455,-0.079182
2022-04-07,0.026468,0.026426,0.022727,0.029077,0.023043,0.023944,0.021402,0.026191,0.030626,0.019183
2022-04-08,-0.042396,-0.038603,-0.036157,-0.046266,-0.042373,-0.037914,-0.044845,-0.042349,-0.044902,-0.044272


In [13]:
daily_returns = pd.concat([mcw_faangm_daily_returns, crypto_daily_returns], axis = 1, join = 'inner')
daily_returns

,faangm,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,-0.012035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,-0.034858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,0.031092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,0.001825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,0.016116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.024245,-0.018089,-0.012366,-0.013260,-0.010834,-0.013602,-0.010075,-0.012517,-0.015033,-0.015440,-0.007345
2022-04-05,-0.017957,-0.020209,-0.013931,0.000436,-0.025822,-0.015793,-0.021087,-0.011304,-0.021187,-0.019172,-0.021287
2022-04-06,-0.028908,-0.088983,-0.086562,-0.093564,-0.088175,-0.084643,-0.076079,-0.089983,-0.083772,-0.092455,-0.079182
2022-04-07,0.000003,0.026468,0.026426,0.022727,0.029077,0.023043,0.023944,0.021402,0.026191,0.030626,0.019183
